In [221]:
import torch
import torchvision
from torch import nn 
from torch import optim
from torch.autograd import Variable
from IPython import display
from torchvision.transforms import ToTensor, Lambda

In [210]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

⦁	Нахождение сложной производной.
Найдите производную по x от функции:

При этом надо пользоваться встроенным в PyTorch autograd

In [74]:
def find_x_derivative(x, y):
    
    x=torch.tensor(x,dtype=torch.float,requires_grad=True)
    y=torch.tensor(y)
    
    f=torch.sin(torch.tan(x)*(x**2/y)+torch.log(torch.exp(-x**2+3)+y*x**3))*torch.tan(torch.exp(x**9)*x**2)
    f.backward()
    return(x.grad)

⦁Нахождение косинусной близости
Вам даны две матрицы A и B. Необходимо посчитать косинусную близость между строчками матрицы A и столбцами матрицы B. Ответ - матрица чисел, где номер строки - номер строки из матрицы А, а номер столбца - номер столбца из В, от которых бралась косинусная близость.
Напомним, что косинусная близость двух векторов - косинус угла между ними. В n-мерном пространстве косинус угла между векторами удобнее всего через скалярное произведение:
 
(Наша операция очень похожа на умножение матриц)
(Выполнять задание следует по представленной формуле)


In [202]:
def cosin(x,y):
    return ((x*y).sum()/((x*x).sum().sqrt()*(y*y).sum().sqrt()))

⦁	Обучение своей нейронной сети

Взять датасет Fashion-Mnist, построить свою нейронную сеть cо сверточными слоями и обучить эту нейронную сеть пользуясь материалами из вебинара. 
Архитектура нейронной сети должна быть следующая:
⦁	Сначала должны идти два блока в которых должны присутствовать следующие слои:
⦁	два подряд идущие сверточных слоя(после каждого сверточного слоя идет активация Relu)
⦁	И в конце каждого блока слой макс пулинга
⦁	в конце должны идти 3 линейных слоя.
При этом можете поиграться с разными оптимизаторами.
В качестве результата прошу приложить скрин с картинками и предсказанными классами.

In [220]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [658]:
training_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))

)

test_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [659]:
train_loader = torch.utils.data.DataLoader(
    training_data, 
    batch_size=64,
    shuffle=False)
test_loader = torch.utils.data.DataLoader(
    test_data, 
    batch_size=4,
    shuffle=False)

In [660]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

In [677]:
import torch.nn.functional as F


class TF(nn.Module):
    def __init__(self):
        super(TF, self).__init__()
        self.flatten = nn.Flatten()
        self.conv1 = nn.Conv1d(28, 56, 3)
        self.conv2 = nn.Conv1d(56, 112, 3)
        self.pool = nn.MaxPool1d(2, 2)
        self.fc1 = nn.Linear(560, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.soft=nn.LogSoftmax()
       

    def forward(self, x):
        x = x.view(-1, 28,28)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3
        x = F.log_softmax(x)
        return x


In [678]:
tf=TF().to('cuda')

In [679]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(tf.parameters())

In [680]:
for epoch in range(10):  # 10 эпох 
    running_loss = 0.0 # Для вывода процесса обучения
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]
        inputs, labels = inputs.cuda(), labels.cuda()

        # Обнуляем градиент
        optimizer.zero_grad()
    
        # Делаем предсказание
        outputs = tf(inputs)

        # Рассчитываем лосс-функцию
        loss = criterion(outputs, labels)
        # Делаем шаг назад по лоссу
        loss.backward()
        # Делаем шаг нашего оптимайзера
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Training is finished!')

AttributeError: 'Linear' object has no attribute 'dim'